let us start by importing the required functions

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
# import statsemodels.api as sm
import SALib
import seaborn as sns
import mpl_toolkits.axisartist as AA
import mpld3
import scipy as sp
import copy
import matplotlib.ticker as ticker

from ema_workbench import (Model, RealParameter, IntegerParameter, Policy, CategoricalParameter, Constant, TimeSeriesOutcome, ScalarOutcome, perform_experiments, ema_logging, save_results, load_results)
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator
from ema_workbench.connectors.pysd_connector import PysdModel 
from ema_workbench.connectors import vensimDLLwrapper
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.util import load_results
from ema_workbench.util import ema_logging

from ema_workbench.analysis import prim
from ema_workbench.analysis.plotting import lines, multiple_densities, kde_over_time
from ema_workbench.analysis.plotting_util import KDE
from ema_workbench.analysis.plotting_util import determine_time_dimension
from ema_workbench.analysis.pairs_plotting import pairs_scatter

from SALib.sample import saltelli
from SALib.analyze import sobol, morris
from SALib.test_functions import Ishigami

from mpl_toolkits.axes_grid1 import host_subplot

from scipy.stats import linregress

ema_logging.log_to_stderr(ema_logging.INFO)

C:\Users\guri_\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:18: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)


<Logger EMA (DEBUG)>

# Adding Reconversion Open Exploration  

### Policy Short No 20 

In [2]:
vensimModel = VensimModel("Step5Adaptive", model_file="ShortTermPolicyforObjectives_Step5_Full Policies_adaptive.vpm" )

In [3]:
vensimModel.uncertainties = [

            RealParameter("Average Fertility Rate", 0.01, 0.025),
            RealParameter("Average Mortality Rate", 0.00126, 0.00154),
            IntegerParameter("BASE Average Household size", 2, 7),     
            RealParameter("average Immigration percent", 0.01, 0.06),
            RealParameter("Housing Area Per person", 0.000004, 0.000006),
            RealParameter("Floor Area Ratio", 2, 7),
            RealParameter("Average Residential approval period", 0.5, 2),
            RealParameter("Base Green Area commisioned per year", 0.02, 0.06),
            RealParameter("average green area construction period", 0.5, 2),
            RealParameter("Base Economic area Conversion rate", 0.01, 0.06),
            RealParameter("Base Slum reconversion rate", 0.01, 0.06),
            RealParameter("Fraction of people selling houses when leaving city", 0.1, 0.5),
            RealParameter("house construction time", 0.5, 2),
            IntegerParameter("time delay effect uncertainty of spending capacity on housing price", 1, 4),
            RealParameter("Slum dwelling area per person", 0.000004, 0.000006),
            RealParameter("Base residential conversion rate", 0.01, 0.05),
            RealParameter("average Emmigration percent", 0.008, 0.05),
            RealParameter("Relocation Rate", 0.00, 0.02), #for relocation of population on streets
            RealParameter("Average Commercial Approval Period", 0.5, 2),
            RealParameter("Base Green Area Reconversion rate", 0.001, 0.03),
            RealParameter("Average Slum Construction period", 0.1, 0.5),
            RealParameter("Average slum Demolition Period", 0.5, 2),
            RealParameter("uncertainty of average percent of inflation effect", 0.01, 0.08),
            RealParameter("average payment to income ratio for housing", 0.2, 0.5),
            RealParameter("Base Non Economic Reconv rate", 0.01 ,0.05),
            RealParameter("Ratio of Renting to Buying",0.2, 1),
            RealParameter("margin of empty housing",0.1,0.5)
]

vensimModel.levers= [
            RealParameter("Policy High rise Program switch",0,1),
            RealParameter("increase in FAR",1,5),
            RealParameter("FAR implementation year",2030,2050),
    
            RealParameter("Accelerated city growth program switch",0,1),
            RealParameter("effect of green recov",1,5),
            RealParameter("Green reconv policy start",2030,2050),
    
            RealParameter("Policy reconversion of Slums Switch",0,1),
            RealParameter("effect of reconversion rate on Policy",1,5),
            RealParameter("Slum reconv polict start year",2030,2050),
    
            RealParameter("Austerity Policy Program Switch",0,1),
            RealParameter("effect on non resi reconv of policy",1,5),
            RealParameter("reconv policy start year",2030,2050),
            RealParameter("Austerity effect on inflation",1,5),
            RealParameter("Auterity implementation year",2030,2050),
    
            RealParameter("Housing Finance Policy Program Switch",0,1),
            RealParameter("Housing finance effect on household spendings",1,5),
            RealParameter("Housing Finane Policy start year",2030,2050),
    
            RealParameter("Policy Relocation program Switch",0,1),
            RealParameter("effect on relocation rate",1,5),
            RealParameter("additional relocation housing factor",1,5),
            RealParameter("Relocation policy start year",2030,2050),
]

vensimModel.outcomes = [
    TimeSeriesOutcome('Housing Affordability Rating'),
    TimeSeriesOutcome('People with No Settlements'),
    TimeSeriesOutcome('Illegal Slum Population'),
    TimeSeriesOutcome('Urban Non Residential Area'),
    TimeSeriesOutcome('Green Area')
    
]
    
MAXIMIZE = ScalarOutcome.MAXIMIZE   
MINIMIZE = ScalarOutcome.MINIMIZE 

vensimModel.robustness_minimax_maximin = [ScalarOutcome('Housing Affordability Rating' , kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('People with No Settlements', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Illegal Slum Population', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Urban Non Residential Area', kind=MAXIMIZE, function=np.min),
                                          ScalarOutcome('Green Area', kind=MAXIMIZE, function=np.min)
                                          ]

In [4]:
n_scenarios = 50
scenarios = sample_uncertainties(vensimModel, n_scenarios)
nfe = 1000
    
with SequentialEvaluator(vensimModel) as evaluator:
    b = evaluator.robust_optimize(vensimModel.robustness_minimax_maximin, scenarios, nfe=nfe, population_size=25,
                                  epsilons=[0.1,]*len(vensimModel.robustness_minimax_maximin) )

[MainProcess/INFO] performing 50 scenarios * 25 policies * 1 model(s) = 1250 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 417 cases completed
[MainProcess/INFO] 834 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments seq

In [4]:
b.to_csv('./Results_49_solutions_minimax_maximin_Longpolicies5adaptive.csv')

NameError: name 'b' is not defined

In [5]:
df_minimax_maximin = pd.DataFrame.from_csv('./Results_49_solutions_minimax_maximin_Longpolicies5adaptive.csv')

C:\Users\guri_\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


In [6]:
df_minimax_maximin.head()

,Policy High rise Program switch,increase in FAR,FAR implementation year,Accelerated city growth program switch,effect of green recov,Green reconv policy start,Policy reconversion of Slums Switch,effect of reconversion rate on Policy,Slum reconv polict start year,Austerity Policy Program Switch,...,Housing Finane Policy start year,Policy Relocation program Switch,effect on relocation rate,additional relocation housing factor,Relocation policy start year,Housing Affordability Rating,People with No Settlements,Illegal Slum Population,Urban Non Residential Area,Green Area
0,0.056202,1.085716,2040.466749,0.910572,1.282143,2047.341659,0.076298,2.949904,2045.992956,0.882419,...,2030.184601,0.916369,4.495066,1.118139,2043.157927,12.155116,14221969.0,600000.0,27.899767,2.403035
1,0.960957,3.618631,2035.322128,0.792672,4.016218,2038.609752,0.943914,3.563636,2044.976884,0.890060,...,2042.638186,0.910705,4.934046,3.341583,2035.077758,45.852623,58832336.0,600000.0,32.638451,2.762157
2,0.929096,3.720486,2044.807573,0.962653,4.507760,2047.049675,0.953754,2.009093,2044.726291,0.890059,...,2033.769728,0.904615,4.414021,1.025921,2049.004735,12.820715,16964110.0,600000.0,29.262096,1.879820
3,0.056202,1.577048,2040.505335,0.973709,3.815450,2047.327656,0.067135,1.685916,2045.984702,0.410800,...,2030.120036,0.804846,2.427956,1.117685,2043.157927,10.959229,13994200.0,600000.0,25.724293,2.020142
4,0.929096,3.720486,2044.807573,0.962653,4.032547,2047.098561,0.953504,2.009093,2044.726291,0.890059,...,2033.865833,0.904615,4.414021,1.025921,2047.896816,12.881039,17107154.0,600000.0,29.148960,1.955564


In [7]:
import plotly

In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [9]:
# create the parcoord

df_parallel = df_minimax_maximin

data = [
    Parcoords(
        line = dict(color = df_parallel['Housing Affordability Rating'],
                   colorscale = 'Jet',
                   showscale = True,
                   reversescale = False,
                   cmin = df_parallel['Housing Affordability Rating'].min(),
                   cmax = df_parallel['Housing Affordability Rating'].max()),

        dimensions = list([
             dict(label ='People with No Settlements'  ,
                values =df_parallel['People with No Settlements']  ),
            dict(label ='Illegal Slum Population'  ,
                values =df_parallel['Illegal Slum Population']  ),
            dict(label = 'Urban Non Residential Area',
                values = df_parallel['Urban Non Residential Area']),
            dict(label = 'Housing Affordability Rating',
                values = df_parallel['Housing Affordability Rating']),
            dict(label = 'Green Area',
                values = df_parallel['Green Area'])
            
        ])
    )
]


layout = Layout(
    title = "Parallel coordinates plot"

)
fig = Figure(data=data, layout=layout)
iplot(fig)